Итог:

Требуемая по заданию модель была получена, но её точность не идеальна. 
В датасете практически нет ячеек с большим количеством багов, из-за чего модель плохо их определяет. 

1) Из имеющихся данных были взяты релевантные к определению кол-ва багов: репозиторий, имя автора, час загрузки и длина сообщения 

2) Данные были преобразованны в ДФ, 317 строк по 35 параметров

3) Было написанно две модели, параметры которых я подбирал

4) Итоговые метрики моделей: 

Accuracy1 = 0.5929 Root_mean_squared_error1 = 0.2030 и R2_Score1 = 0.51

Accuracy2 = 0.5771 Root_mean_squared_error2 = 0.2035 и R2_Score2 = 0.54

5) Визуально работа модели представленна в 3 снизу ячейке 


In [1]:
from google.colab import drive

In [2]:
drive.mount ('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
np.random.seed (1111)
import matplotlib.pyplot as plt
%matplotlib inline
import keras
from  keras.datasets import mnist
from  keras.models import Model
from  keras.layers import *
from  keras import optimizers
from  sklearn.model_selection import train_test_split
from  keras.utils.np_utils import to_categorical
from sklearn.metrics import r2_score

In [4]:
df_train = pd.read_csv ('/content/drive/MyDrive/Content/АВСОФТ_полная.csv') #импортим тренировочный набор

In [5]:
df_train

,bugs,commit_hash,commit_date,commit_author,commit_message,repository_name
0,4,b28abb42,2020-06-12 14:26:12,Victor,объяснения предсказаний с помощью lime,mlm
1,3,438b5c4d,2020-06-12 11:53:10,Victor,Поставил lime вместо eli5,mlm
2,4,8e6e4f31,2020-06-11 15:19:03,Victor,Скрипт для создния пакетов обновления,mlm
3,3,5e0956a6,2020-06-11 10:42:29,Victor,Сортировка весов по убыванию,mlm
4,7,b9f4b40c,2020-06-11 9:25:19,Victor,не надо конвертировать массив до операции по с...,mlm
...,...,...,...,...,...,...
312,1,5b1ecbcd,2020-03-23 12:54:35,Dabe,setting hostname,sensor
313,2,a3f5b753,2020-03-23 12:30:52,Dabe,heralding default dockerfile,sensor
314,3,e8155942,2020-03-23 11:27:40,Dabe,unnesserity /data path permissions setting,sensor
315,1,e748af51,2020-03-23 11:20:50,Dabe,added trapconductor,sensor


In [6]:
#Автор и Репозиторий это точно важные факторы. Поэтому их мы оставляем в данных, приведя в другой вид. 
df_train = pd.get_dummies(df_train, columns=['repository_name'])
df_train = pd.get_dummies(df_train, columns=['commit_author'])
#for i in range(len(df_train.commit_hash.values)):    #Хеш код перевожу в десятичное число, зависимости от хеша у количества багов нет
#  df_train.commit_hash.values[i] = int(df_train.commit_hash.values[i],16)/10000
del df_train['commit_hash'] #не влияющая на количество багов информация. Я попробовал использовать её в модели и это снизило её точность в ~2 раза. 
for i in range(len(df_train.commit_date.values)):
  p1 = df_train.commit_date.values[i].find(' ')+1
  p2 = df_train.commit_date.values[i].find(':')
  df_train.commit_date.values[i] = int(df_train.commit_date.values[i][p1:p2],10) #Время загрузки комита может влиять на кол-во багов. Используем час загрузки, тк минуты слишком точно.
df_train = pd.get_dummies(df_train, columns=['commit_date'])
df_train.commit_message = df_train.commit_message.str.len() #Логично предположить, что чем длиннее сообщение, тем больше или важнее внессенные в программу изменения. Поэтому я беру длину сообщений. 
df_train

,bugs,commit_message,repository_name_agent,repository_name_conductor,repository_name_dockers,repository_name_mlm,repository_name_sensor,repository_name_standard,commit_author_Alice,commit_author_Bob,...,commit_date_14,commit_date_15,commit_date_16,commit_date_17,commit_date_18,commit_date_19,commit_date_20,commit_date_21,commit_date_22,commit_date_23
0,4,38,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,3,25,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,37,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,3,28,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,7,56,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312,1,16,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
313,2,28,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
314,3,42,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
315,1,19,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
#на всякий случай форматирую данные

df_features = df_train.iloc[:, 1:36]
df_label = df_train.iloc[:, 0]
X_train = df_train.iloc[:, 1:36] 
X_train = X_train.values.reshape(317, 35)
print(X_train.shape)

(317, 35)


In [8]:
#Отрезал Тестовый массив данных от Проверочного
X_train, X_cv, y_train, y_cv = train_test_split(df_features, df_label,
                                                test_size = 0.2,
                                                shuffle=True,
                                                random_state = 1111)

In [10]:
X_train = X_train.astype('float32')
X_cv = X_cv.astype('float32')

In [11]:
num_digits = 13
y_train = to_categorical(y_train, num_digits)
y_cv = to_categorical(y_cv, num_digits)

In [12]:
n_input = 35
n_hidden_1 = 70  
n_hidden_2 = 35
num_digits = 13

In [13]:
#Задаю слои

Inp = Input(shape = (35,))
x = Dense(n_hidden_1, activation='swish', name = 'hidden_layer_1')(Inp)
x = Dense(n_hidden_2, activation='swish', name = 'hidden_layer_2')(x)
output = Dense(num_digits, activation='softmax', name = 'out_layer')(x)

In [14]:
model = Model(Inp, output)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 35)]              0         
                                                                 
 hidden_layer_1 (Dense)      (None, 70)                2520      
                                                                 
 hidden_layer_2 (Dense)      (None, 35)                2485      
                                                                 
 out_layer (Dense)           (None, 13)                468       
                                                                 
Total params: 5,473
Trainable params: 5,473
Non-trainable params: 0
_________________________________________________________________


In [15]:
learning_rate = 0.1
training_epochs = 40
batch_size = 25
rmsprop = tf.keras.optimizers.RMSprop(learning_rate)

In [16]:
model.compile(loss='categorical_crossentropy',
              optimizer = 'rmsprop',
              metrics=['accuracy', tf.keras.metrics.RootMeanSquaredError()]) 

In [17]:
history1 = model.fit(X_train, y_train,
                      batch_size = batch_size,
                      epochs = training_epochs,
                      verbose = 2,
                      validation_data=(X_cv, y_cv))

Epoch 1/40
11/11 - 1s - loss: 2.8663 - accuracy: 0.3004 - root_mean_squared_error: 0.2620 - val_loss: 2.5636 - val_accuracy: 0.2656 - val_root_mean_squared_error: 0.2590 - 1s/epoch - 108ms/step
Epoch 2/40
11/11 - 0s - loss: 2.1799 - accuracy: 0.3439 - root_mean_squared_error: 0.2525 - val_loss: 2.2541 - val_accuracy: 0.3125 - val_root_mean_squared_error: 0.2544 - 69ms/epoch - 6ms/step
Epoch 3/40
11/11 - 0s - loss: 1.9598 - accuracy: 0.3874 - root_mean_squared_error: 0.2453 - val_loss: 2.0867 - val_accuracy: 0.3125 - val_root_mean_squared_error: 0.2479 - 53ms/epoch - 5ms/step
Epoch 4/40
11/11 - 0s - loss: 1.8687 - accuracy: 0.3953 - root_mean_squared_error: 0.2421 - val_loss: 2.0107 - val_accuracy: 0.4219 - val_root_mean_squared_error: 0.2484 - 48ms/epoch - 4ms/step
Epoch 5/40
11/11 - 0s - loss: 1.7930 - accuracy: 0.4111 - root_mean_squared_error: 0.2394 - val_loss: 1.9650 - val_accuracy: 0.2812 - val_root_mean_squared_error: 0.2475 - 68ms/epoch - 6ms/step
Epoch 6/40
11/11 - 0s - loss: 

In [18]:
n_input = 35
n_hidden_1 = 70
n_hidden_2 = 35
n_hidden_3 = 20
num_digits = 13

In [19]:
Inp = Input(shape=(35,))
x = Dense(n_hidden_1, activation='swish', name = 'hidden_layer_1')(Inp)
x = Dense(n_hidden_2, activation='swish', name = 'hidden_layer_2')(x)
x = Dense(n_hidden_3, activation='swish', name = 'hidden_layer_3')(x)
output = Dense(num_digits, activation='softmax', name = 'out_layer')(x)

In [20]:
learning_rate = 0.1
training_epochs = 40
batch_size = 25
adam = tf.keras.optimizers.Adam(learning_rate)
model2 = Model(Inp, output)

In [21]:
model2.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 35)]              0         
                                                                 
 hidden_layer_1 (Dense)      (None, 70)                2520      
                                                                 
 hidden_layer_2 (Dense)      (None, 35)                2485      
                                                                 
 hidden_layer_3 (Dense)      (None, 20)                720       
                                                                 
 out_layer (Dense)           (None, 13)                273       
                                                                 
Total params: 5,998
Trainable params: 5,998
Non-trainable params: 0
_________________________________________________________________


In [22]:
model2.compile(loss='categorical_crossentropy',
               optimizer='adam',
               metrics=['accuracy', tf.keras.metrics.RootMeanSquaredError()])

In [23]:
history2 = model2.fit(X_train, y_train,
                      batch_size = batch_size,
                      epochs = training_epochs,
                      verbose = 2,
                      validation_data=(X_cv, y_cv))

Epoch 1/40
11/11 - 1s - loss: 2.9219 - accuracy: 0.1739 - root_mean_squared_error: 0.2721 - val_loss: 2.3623 - val_accuracy: 0.1250 - val_root_mean_squared_error: 0.2617 - 874ms/epoch - 79ms/step
Epoch 2/40
11/11 - 0s - loss: 2.2886 - accuracy: 0.2253 - root_mean_squared_error: 0.2599 - val_loss: 2.2038 - val_accuracy: 0.3750 - val_root_mean_squared_error: 0.2582 - 57ms/epoch - 5ms/step
Epoch 3/40
11/11 - 0s - loss: 2.1323 - accuracy: 0.3992 - root_mean_squared_error: 0.2554 - val_loss: 2.0655 - val_accuracy: 0.4375 - val_root_mean_squared_error: 0.2531 - 72ms/epoch - 7ms/step
Epoch 4/40
11/11 - 0s - loss: 1.9652 - accuracy: 0.4111 - root_mean_squared_error: 0.2489 - val_loss: 1.9727 - val_accuracy: 0.4531 - val_root_mean_squared_error: 0.2469 - 76ms/epoch - 7ms/step
Epoch 5/40
11/11 - 0s - loss: 1.8548 - accuracy: 0.4111 - root_mean_squared_error: 0.2429 - val_loss: 1.9330 - val_accuracy: 0.4062 - val_root_mean_squared_error: 0.2429 - 48ms/epoch - 4ms/step
Epoch 6/40
11/11 - 0s - loss

In [24]:
df_test = pd.read_csv ('/content/drive/MyDrive/Content/АВСОФТ_тест.csv')
df_test.head()

,bugs,commit_hash,commit_date,commit_author,commit_message,repository_name
0,1,f5676284,2020-06-04 21:19:11,Wendy,fix oups,conductor
1,9,af7c2776,2020-03-08 17:46:38,Victor,Saving unknown labels on emergency saving,mlm
2,2,a6b4a5c5,2020-04-17 13:27:01,Mallory,added service hashsum,conductor
3,6,831a8b6e,2020-04-06 16:36:00,Mallory,Popen service enabling/disabling; removing ima...,conductor
4,6,bfa1152f,2020-03-08 15:38:30,Victor,JsonMaster as function separator,mlm


In [25]:
#привожу дфтест в вид дфтрейна, чтобы визуализировать результаты модели

df_test = pd.get_dummies(df_test, columns=['repository_name'])
df_test = pd.get_dummies(df_test, columns=['commit_author'])
del df_test['commit_hash']
for i in range(len(df_test.commit_date.values)):
  p1 = df_test.commit_date.values[i].find(' ')+1
  p2 = df_test.commit_date.values[i].find(':')
  df_test.commit_date.values[i] = int(df_test.commit_date.values[i][p1:p2],10) 
df_test = pd.get_dummies(df_test, columns=['commit_date'])
df_test.commit_message = df_test.commit_message.str.len() 

df_test = df_test.reindex(df_train.columns , axis = 'columns', fill_value = 0)

df_test.head()

,bugs,commit_message,repository_name_agent,repository_name_conductor,repository_name_dockers,repository_name_mlm,repository_name_sensor,repository_name_standard,commit_author_Alice,commit_author_Bob,...,commit_date_14,commit_date_15,commit_date_16,commit_date_17,commit_date_18,commit_date_19,commit_date_20,commit_date_21,commit_date_22,commit_date_23
0,1,8,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,9,41,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,2,21,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,74,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,6,32,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [26]:
X_vis = df_test.iloc[:, 1:37]
y_vis = df_test.iloc[:, 0]

In [27]:
##########################################
#ВИЗУАЛИЗАЦИЯ РЕЗУЛЬТАТА ОБУЧЕНИЯ МОДЕЛЕЙ#
##########################################

test_pred = pd.DataFrame(model.predict(X_vis, batch_size=30))
test_pred = pd.DataFrame(test_pred.idxmax(axis = 1))
test_pred.index.name = 'Index'
test_pred = test_pred.rename(columns = {0: 'Model1 pred'})
test_pred2 = pd.DataFrame(model2.predict(X_vis, batch_size=30))
test_pred2 = pd.DataFrame(test_pred2.idxmax(axis = 1))
test_pred["Model2 pred"] = '0'
test_pred["Model2 pred"] = test_pred2[0]
test_pred["Labels"] = y_vis
print(test_pred)

       Model1 pred  Model2 pred  Labels
Index                                  
0                1            1       1
1                4            4       9
2                2            2       2
3                5            4       6
4                4            4       6
5                4            4       5
6                2            2       2
7                4            5       5
8                4            4       4
9                2            2       1
10               3            4       8
11               4            4       6
12               2            3       1
13               2            3       3
14               2            2       1
15               2            1       2
16               4            4       4
17               1            1       1
18               4            4       4
19               2            2       2
20               2            3       2
21               1            1       1
22               2            3       3


In [28]:
r2_score(test_pred["Labels"], test_pred["Model1 pred"])

0.5099431818181818

In [29]:
r2_score(test_pred["Labels"], test_pred["Model2 pred"])

0.5454545454545454